In [1]:
import numpy as np
import tensorflow as tf
from moviepy.editor import VideoFileClip, ImageSequenceClip
import scipy
import matplotlib.pyplot as plt
import os
from glob import glob

## Load images and paste predicted labels

In [2]:
IMAGE_DIR = './images/stream1'
image_paths = sorted(glob(os.path.join(IMAGE_DIR, 'CameraRGB', '*.png')))
load_pb_path = './inference_pb/fused_graph.pb'
tf.reset_default_graph()
with tf.Session(graph=tf.Graph()) as sess:
    gd = tf.GraphDef()
    with tf.gfile.Open(load_pb_path, 'rb') as f:
        data = f.read()
        gd.ParseFromString(data)
    tf.import_graph_def(gd, name='')
    graph = tf.get_default_graph()
    input_image = graph.get_tensor_by_name('input_image:0')
    softmax = graph.get_tensor_by_name('softmax:0')
    crop_up = 102
    crop_down = 518
    image_shape = [600,800,3]
    results = []
    f = 1
    for image_file in image_paths:
        rgb_frame = scipy.misc.imread(image_file)
        crop_frame = rgb_frame[crop_up:crop_down,:,:]
        crop_frame = np.expand_dims(crop_frame,0)
        out = sess.run(softmax, feed_dict={input_image: crop_frame})[0]
        out_arg = np.argmax(out,axis = 2)
        # Look for red cars :)
        binary_car_result = np.where(out_arg == 1,1,0)
        binary_car_result = np.concatenate((np.zeros([crop_up,image_shape[1]]),binary_car_result), axis=0)
        binary_car_result = np.concatenate((binary_car_result, np.zeros([image_shape[0]-crop_down,image_shape[1]])), axis=0)
        mask_car = np.dot(np.expand_dims(binary_car_result, axis=2), np.array([[0, 255, 0, 127]]))
        mask_car = scipy.misc.toimage(mask_car, mode="RGBA")
        # Look for road :)
        binary_road_result = np.where(out_arg == 2,1,0)
        binary_road_result = np.concatenate((np.zeros([crop_up,image_shape[1]]),binary_road_result), axis=0)
        binary_road_result = np.concatenate((binary_road_result, np.zeros([image_shape[0]-crop_down,image_shape[1]])), axis=0)
        mask_road = np.dot(np.expand_dims(binary_road_result, axis=2), np.array([[0, 0, 255, 127]]))
        mask_road = scipy.misc.toimage(mask_road, mode="RGBA")
        frame_merge = scipy.misc.toimage(rgb_frame)
        frame_merge.paste(mask_car, box=None, mask=mask_car)
        frame_merge.paste(mask_road, box=None, mask=mask_road)
        results.append(np.array(frame_merge))
        print('Processing frame: {0}/{1}'.format(f,len(image_paths)),end='\r')
        f +=1

## Convert image sequence to video

In [3]:
output_path = os.path.join('./videos','test_video1.mp4')
output_clip = ImageSequenceClip(results, fps=25)
output_clip.write_videofile(output_path, audio=False)

[MoviePy] >>>> Building video ./videos/test_video1.mp4
[MoviePy] Writing video ./videos/test_video1.mp4


100%|██████████| 479/479 [00:03<00:00, 151.59it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./videos/test_video1.mp4 



In [4]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_path))